## Hi This is my data preparation worksheet

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
      raw_text = f.read()

print("Total number of characters: ", len(raw_text))
raw_text[:99]

Total number of characters:  20479


'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no '

In [4]:
import re

In [9]:
text = "Hello I am just trying this out!"
result = re.split(r'(\s)', text)
print(result)

['Hello', ' ', 'I', ' ', 'am', ' ', 'just', ' ', 'trying', ' ', 'this', ' ', 'out!']


In [10]:
result = re.split(r'[,.]|(\s)', text)
print(result)

['Hello', ' ', 'I', ' ', 'am', ' ', 'just', ' ', 'trying', ' ', 'this', ' ', 'out!']


In [12]:
result = [item for item in result if item.strip()]
result

['Hello', 'I', 'am', 'just', 'trying', 'this', 'out!']

This is a simple example of how a tokenizer works

In [13]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)

In [15]:
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [18]:
print(len(preprocessed))

4690


Convert Tokens into Token IDs

In [19]:
allwords = sorted(set(preprocessed))

In [21]:
vocab_size = len(allwords)
vocab_size

1130

In [23]:
vocab = {token:integer for integer,token in enumerate(allwords)}

In [27]:
for i,item in enumerate(vocab.items()):
    print(i,"->",item)
    if i>=20:
        break

0 -> ('!', 0)
1 -> ('"', 1)
2 -> ("'", 2)
3 -> ('(', 3)
4 -> (')', 4)
5 -> (',', 5)
6 -> ('--', 6)
7 -> ('.', 7)
8 -> (':', 8)
9 -> (';', 9)
10 -> ('?', 10)
11 -> ('A', 11)
12 -> ('Ah', 12)
13 -> ('Among', 13)
14 -> ('And', 14)
15 -> ('Are', 15)
16 -> ('Arrt', 16)
17 -> ('As', 17)
18 -> ('At', 18)
19 -> ('Be', 19)
20 -> ('Begin', 20)
